In [ ]:
#Importing libraries to connect to transcend and open table in the form of Teradata DataFrame
from teradataml.context.context import *
from teradataml.dataframe.dataframe import DataFrame
import getpass
import pandas as pd
import numpy as np
from teradataml.dataframe.fastload import fastload
from teradatasqlalchemy.types import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.ensemble import RandomForestClassifier
import tempfile
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer

In [ ]:
#Creating a connection to the Transcend
create_context(host='tdprd.td.teradata.com', 
username='ka255037', password=getpass.getpass(), 
temp_database_name ='ka255037', logmech='LDAP')

In [ ]:
Telco_df = DataFrame.from_query("select * from tm122469.Telco_Churn_Prep sample 99999")
Telco_df = Telco_df.to_pandas()
pd.set_option('display.max_columns', None)
Telco_df.head(10)

In [ ]:
features = ['Gender', 'SeniorCitizen', 'Partner', 'Dependents',
                     'TenureMonths',
                     'PhoneService',
                     'PaperlessBilling',
                     'MonthlyCharges',
                     'TotalCharges',
                     'CLTV',
                     'Age',
                     'Under30',
                     'Married',
                     'NumberOfDependents',
                     'ReferredAFriend',
                     'NumberOfReferrals',
                     'AvgMonthlyLongDistanceCharges',
                     'AvgMonthlyGBDownload',
                     'DeviceProtectionPlan',
                     'PremiumTechSupport',
                     'StreamingMusic',
                     'UnlimitedData',
                     'TotalRefunds',
                     'TotalExtraDataCharges',
                     'TotalLongDistanceCharges',
                     'TotalRevenue',
                     'SatisfactionScore',
                     'MultipleLines_No',
                     'MultipleLines_Yes',
                     'InternetService_DSL',
                     'InternetService_FiberOptic',
                     'OnlineSecurity_No',
                     'OnlineSecurity_Yes',
                     'OnlineBackup_No',
                     'OnlineBackup_Yes',
                     'DeviceProtection_No',
                     'DeviceProtection_Yes',
                     'TechSupport_No',
                     'TechSupport_Yes',
                     'StreamingTV_No',
                     'StreamingTV_Yes',
                     'StreamingMovies_No',
                     'StreamingMovies_Yes',
                     'Contract_OneYear',
                     'Contract_TwoYear',
                     'PaymentMethod_AutoBankTransfer',
                     'PaymentMethod_AutoCreditCard',
                     'PaymentMethod_ECheck',
                     'InternetType_Cable',
                     'InternetType_DSL',
                     'InternetType_FiberOptic',
                     'Offer_OfferA',
                     'Offer_OfferB',
                     'Offer_OfferC',
                     'Offer_OfferD',
                     'Offer_OfferE']

 
target = "ChurnValue"

In [ ]:
telco_X = Telco_df[features]
telco_y = Telco_df[target]

In [ ]:
#pipeline_obj = Pipeline([
#    ("mapping", DataFrameMapper([
#    (features, StandardScaler()) , 
#    (features,SimpleImputer(missing_values=np.nan, strategy='mean'))
#    ])),
#    ("rfc", RandomForestClassifier(n_estimators = 100))
#])

#pipeline_obj.fit(Telco_df[features],Telco_df[target])

In [ ]:
# Remove the temp directory code and specify your system's path if you want to save and read the model from your hard drive
#from nyoka import skl_to_pmml
#temp_dir = tempfile.TemporaryDirectory()
#skl_to_pmml(pipeline_obj,features,target,"telco_churn_byom.pmml")

In [ ]:
classifier = RandomForestClassifier(n_estimators=10)
pipeline_model = PMMLPipeline([('classifier', 
classifier)]).fit(telco_X, telco_y)
from sklearn2pmml import sklearn2pmml
sklearn2pmml(pipeline_model, 'randomforest.pmml', with_repr = True)

In [ ]:
#model_bytes = open(f"{temp_dir.name}/telco_churn_byom.pmml", "rb").read()
#temp_dir.cleanup()
model_bytes = open("randomforest.pmml", "rb").read()

In [ ]:
model_bytes

In [ ]:
#Install Teradata sqlalchemy as well or else this is not working
import sqlalchemy
import getpass
password = getpass.getpass()
connection_text = 'teradatasql://whomooz/?user={}&password={}&host={}&database={}&logmech={}'
connection_text = connection_text.format("ka255037",password,"tdprd.td.teradata.com","ADLTIT_Kumar_Demos","LDAP")
eng = sqlalchemy.create_engine(connection_text)

In [ ]:
#Create model table if it doesn’t exist
create_cmd = '''CREATE SET TABLE pmml_models (
  model_id VARCHAR (30),
  model BLOB
)
PRIMARY INDEX (model_id);'''
eng.execute(create_cmd)

In [ ]:
eng.execute("delete from pmml_models  where model_id = 'telco_churn_byom'")
modelname_param = "telco_churn_byom"
insert_model = f"insert into ADLTIT_Kumar_Demos.pmml_models (model_id, model) values(?,?);"
eng.execute(insert_model, modelname_param, model_bytes)

In [ ]:
qp_drop = "drop table ADLTIT_Kumar_Demos.telco_pred_results;"
eng.execute(qp_drop)
q_p = '''create table ADLTIT_Kumar_Demos.telco_pred_results as (select * from TRNG_BYOM.PMMLPredict( on (select * from "tm122469"."Telco_Churn_Prep") on (select * from pmml_models where model_id = 'telco_churn_byom') DIMENSION using Accumulate('*') ) td sample 100) with data;'''
telco_pred = eng.execute(q_p)

In [ ]:
telco_pred = DataFrame.from_query("select * from ADLTIT_Kumar_Demos.telco_pred_results;")
telco_pred = telco_pred.to_pandas()
pd.set_option('display.max_columns', None)
telco_pred.head(10)